# Bottle data in bats


In [24]:
import pandas as pd
import numpy as np
import xarray as xr
import cf_xarray.units  # noqa: F401
import pint_xarray  # noqa: F401
import json

pd.options.plotting.backend = "plotly"

## Load raw data

---


In [25]:
raw_data = pd.read_csv("../../1_raw/bats_zooplankton.csv", index_col=0)
raw_metadata = pd.read_csv("../../1_raw/bats_zooplankton_meta.csv", index_col=0)

In [26]:
raw_metadata = raw_metadata.set_index("Variable")

raw_data["time"] = pd.to_datetime(raw_data["time"])
raw_data["sieve_size"] = raw_data["sieve_size"].astype("category")
raw_data = raw_data.drop(columns=["Cruise_ID", "time_out", "duration_minutes", "UNOLS"])

## Clean data

---


In [27]:
fig = (
    raw_data.isna()
    .sum()
    .plot(
        kind="bar",
        title=f"Missing values per column for a total of {len(raw_data)} entries",
        labels=dict(
            index="Column",
            value="Number of missing values",
        ),
        # log y-axis
        log_y=True,
    )
)
# Rotate x-axis labels by 45 degrees
fig.update_xaxes(tickangle=-45)
# multiply the figure ratio of height by 2
fig.update_layout(height=500)
# Remove legend
fig.update_layout(showlegend=False)
fig.show()

## Manage index

---


In [28]:
raw_data["time_in"] = (
    raw_data["time_in"]
    .astype(str)
    .apply(lambda x: x.zfill(4))
    .apply(lambda x: x[:2] + ":" + x[2:] + ":00")
    .apply(pd.to_timedelta)
)
raw_data["time"] = raw_data["time"] + raw_data["time_in"]
raw_data = raw_data.drop(columns=["time_in"])

In [29]:
raw_data["time"].value_counts().sort_index().plot(
    title="Number of entries per time",
    labels=dict(
        index="Time",
        value="Number of entries",
    ),
)

In [30]:
raw_data.time.dt.hour.hist(bins=24, title="Number of entries per hour")

In [31]:
raw_data["is_day"] = raw_data.time.dt.hour.isin(range(6, 18))
raw_data["time"] = raw_data.time.dt.floor("D")
raw_data.head(2)

,time,lat,lon,depth,volume_water,sieve_size,wet_weight,dry_weight,wet_weight_vol_water_ratio,dry_weight_vol_water_ratio,total_wet_weight_volume_all_size_fractions_ratio,total_dry_weight_volume_all_size_fractions_ratio,wet_weight_vol_water_ratio_200m_depth,dry_weight_vol_water_ratio_200m_depth,total_wet_weight_volume_all_size_fractions_200m,total_dry_weight_volume_all_size_fractions_200m,is_day
0,1995-05-10,31.6625,-64.169167,200.0,433.408,200.0,1520.8,242.8,3.51,0.56,22.9,3.49,701.79,112.04,4580.26,697.82,False
1,1995-05-10,31.6625,-64.169167,200.0,433.408,500.0,2156.0,300.4,4.97,0.69,22.9,3.49,994.91,138.62,4580.26,697.82,False


In [32]:
raw_data["lat"].astype(float).plot.hist(nbins=100, title="Latitude")

In [33]:
raw_data["lat"] = np.full_like(raw_data["lat"], 31.6)

In [34]:
raw_data["lon"].astype(float).plot.hist(nbins=100, title="Latitude")

In [35]:
raw_data["lon"] = np.full_like(raw_data["lon"], -64.2)

In [36]:
raw_data["depth"].astype(float).plot.hist(
    nbins=100, title="Depth", labels=dict(value="Depth (m)")
)

Then aggregate data by 50 meters.


In [37]:
# group depth by 50m bins
raw_data["depth"] = pd.cut(
    raw_data["depth"],
    bins=[0, 50, 100, 150, 200, 250, 300, 350, 400],
    labels=[50, 100, 150, 200, 250, 300, 350, 400],
    right=False,
)

In [38]:
raw_data["depth"].astype(float).plot.hist(title="Depth", labels=dict(value="Depth (m)"))

## Produce preprocessed data

---


Group by tow and use the mean values. Use the size of the frac rather than the flag to represent the size of the zooplankton.


In [39]:
# This show that there are duplicates index

try:
    raw_data.set_index(
        ["time", "depth", "lat", "lon", "sieve_size"], verify_integrity=True
    )
except ValueError as e:
    print(e)

Index has duplicate keys: MultiIndex([('1995-05-11', 250, 31.6, -64.2,  200.0),
            ('1995-05-11', 250, 31.6, -64.2,  500.0),
            ('1995-05-11', 250, 31.6, -64.2, 1000.0),
            ('1995-05-11', 250, 31.6, -64.2, 2000.0),
            ('1995-05-11', 250, 31.6, -64.2, 5000.0),
            ('1995-07-11', 200, 31.6, -64.2,  200.0),
            ('1995-07-11', 200, 31.6, -64.2,  500.0),
            ('1995-07-11', 200, 31.6, -64.2, 1000.0),
            ('1995-07-11', 200, 31.6, -64.2, 2000.0),
            ('1995-07-11', 200, 31.6, -64.2, 5000.0),
            ...
            ('2022-10-12', 200, 31.6, -64.2,  200.0),
            ('2022-10-12', 200, 31.6, -64.2,  500.0),
            ('2022-10-12', 200, 31.6, -64.2, 1000.0),
            ('2022-10-12', 200, 31.6, -64.2, 2000.0),
            ('2022-10-12', 200, 31.6, -64.2, 5000.0),
            ('2022-12-13', 150, 31.6, -64.2,  200.0),
            ('2022-12-13', 150, 31.6, -64.2,  500.0),
            ('2022-12-13', 150, 31.6, -6

In [40]:
raw_data["depth"] = raw_data["depth"].astype(float)
# Convert to millimeters
raw_data["sieve_size"] = raw_data["sieve_size"].astype(float) / 1000

In [41]:
raw_data = raw_data.rename(columns={"lat": "latitude", "lon": "longitude"})

preprocessed_data = xr.Dataset.from_dataframe(
    raw_data.groupby(
        ["time", "is_day", "depth", "latitude", "longitude", "sieve_size"]
    ).mean()
)

for var in preprocessed_data:
    attrs = raw_metadata.loc[var].to_dict()
    # set all attrs keys to lowercase
    attrs = {k.lower(): v for k, v in attrs.items()}
    # transform unit key to units
    attrs["units"] = attrs.pop("unit")
    preprocessed_data[var].attrs = attrs

preprocessed_data["time"].attrs = {
    "standard_name": "time",
    "long_name": "time",
    "axis": "T",
}
preprocessed_data["is_day"].attrs = {
    "flag_values": f"{[True, False]}",
    "flag_meanings": "day night",
    "standard_name": "is_day",
    "long_name": "Is day",
    "description": "Flag to indicate if the time is during the day or night",
}
preprocessed_data["latitude"].attrs = {
    "standard_name": "latitude",
    "long_name": "latitude",
    "axis": "Y",
    "units": "degrees_north",
}
preprocessed_data["longitude"].attrs = {
    "standard_name": "longitude",
    "long_name": "longitude",
    "axis": "X",
    "units": "degrees_east",
}
preprocessed_data["depth"].attrs = {
    "standard_name": "depth",
    "long_name": "depth",
    "axis": "Z",
    "units": "m",
}
preprocessed_data["sieve_size"].attrs = {
    "standard_name": "size",
    "long_name": "Sieve size",
    "units": "mm",
}

preprocessed_data

<xarray.Dataset> Size: 3MB
Dimensions:                                           (time: 541, is_day: 2,
                                                       depth: 7, latitude: 1,
                                                       longitude: 1,
                                                       sieve_size: 5)
Coordinates:
  * time                                              (time) datetime64[ns] 4kB ...
  * is_day                                            (is_day) bool 2B False ...
  * depth                                             (depth) float64 56B 50....
  * latitude                                          (latitude) float64 8B 31.6
  * longitude                                         (longitude) float64 8B ...
  * sieve_size                                        (sieve_size) float64 40B ...
Data variables:
    volume_water                                      (time, is_day, depth, latitude, longitude, sieve_size) float64 303kB ...
    wet_weight                                        (time, is_day, depth, latitude, longitude, sieve_size) float64 303kB ...
    dry_weight                                        (time, is_day, depth, latitude, longitude, sieve_size) float64 303kB ...
    wet_weight_vol_water_ratio                        (time, is_day, depth, latitude, longitude, sieve_size) float64 303kB ...
    dry_weight_vol_water_ratio                        (time, is_day, depth, latitude, longitude, sieve_size) float64 303kB ...
    total_wet_weight_volume_all_size_fractions_ratio  (time, is_day, depth, latitude, longitude, sieve_size) float64 303kB ...
    total_dry_weight_volume_all_size_fractions_ratio  (time, is_day, depth, latitude, longitude, sieve_size) float64 303kB ...
    wet_weight_vol_water_ratio_200m_depth             (time, is_day, depth, latitude, longitude, sieve_size) float64 303kB ...
    dry_weight_vol_water_ratio_200m_depth             (time, is_day, depth, latitude, longitude, sieve_size) float64 303kB ...
    total_wet_weight_volume_all_size_fractions_200m   (time, is_day, depth, latitude, longitude, sieve_size) float64 303kB ...
    total_dry_weight_volume_all_size_fractions_200m   (time, is_day, depth, latitude, longitude, sieve_size) float64 303kB ...

## Final plot

---


In [45]:
raw_data.groupby(["time", "is_day", "latitude", "longitude", "depth"]).mean().plot.box()

## Export preprocessed data

---


In [46]:
try:
    preprocessed_data.pint.quantify()
except Exception as e:
    print(e)
    print("Some units cannot be quantified and are only here for information.")

In [47]:
preprocessed_data.to_zarr("../../2_processed/bats_zooplankton.zarr", mode="w")